In [ ]:
try:
  import unidecode
except ModuleNotFoundError:
  !pip install pyogrio==0.7.2 geopandas==0.14.3 unidecode

  Attempting uninstall: geopandas
    Found existing installation: geopandas 0.14.4
    Uninstalling geopandas-0.14.4:
      Successfully uninstalled geopandas-0.14.4


In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
from IPython.core.magic import register_cell_magic
import glob
import pickle
from unidecode import unidecode
import numpy as np
from google.colab import drive
import ee
import geemap
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import geopandas as gpd
import pyproj
import pyarrow

In [ ]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')

gpd.options.io_engine = "pyogrio"
os.environ["PYOGRIO_USE_ARROW"] = "1"

sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils

@register_cell_magic
def skip(line, cell):
    return

interim_path = "/content/drive/MyDrive/CAFO_data/forTraining/interim_files/"
shp_path = "/content/drive/MyDrive/CAFO_data/"
out_path = "/content/drive/MyDrive/CAFO_data/forTraining/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sentinel_bands = ['B4', 'B3', 'B2'] # mainly to reduce data volume/get max resolution
                                    # keep bands in this order

training_image_radius = 240 # m, to obtain (approx) 48 x ?? pixel images

In [ ]:
# Get the dfs containing the farm and not-farm buildings/locations selected for
# training. The notebook will output images and metadata for all datasets
# included here (combined).

chl_bldgs = pd.read_pickle(interim_path+'Chile_bldgs_cleaned.pkl')
mex_bldgs = pd.read_pickle(interim_path+'Mexico_bldgs_cleaned.pkl')

In [ ]:
# Get the ADMx-level boundaries for each country. This is just to break things
# up into chunks that don't make Earth Engine barf. It's OK to do Mexico state-
# by=state (ADM2), but Chile needs to be run at the Comuna level (ADM3) for
# reasons I don't quite understand.

mex_areas = gpd.read_file(f"{shp_path}Mexico/shapefiles/mex_admbnda_adm2_govmex_20210618.shp")
chl_areas = gpd.read_file(f"{shp_path}Chile/shapefiles/chl_admbnda_adm3_bcn_20211008.shp")\
                 .set_crs("EPSG:4326")
# Don't know why setting CRS is necessary for CHL but not MEX

In [ ]:
# Identify the administrative regions that contain farm/not-farm locations,
# because we don't want to bother iterating over ones with no farms

def narrow_down(bldgs_df, boundaries_df, adm_no):

  bldgs_df.loc[:, 'location_geom'] = bldgs_df.loc[:, 'geometry']
  joined = boundaries_df.sjoin(bldgs_df, how='inner', predicate='intersects')

  # create a df containing only boundaries containing buildings
  bounds_w_data = joined.drop_duplicates(subset=[f'ADM{adm_no}_ES'])

  # create a version of the buildings df in which ADMx_ES is identified
  bldgs_df_2 = joined.drop(columns=['geometry'])\
                     .rename(columns={'location_geom': 'geometry'})\
                     .set_geometry('geometry')

  # drop extraneous columns, but keep admin areas as they will be useful for
  # defining held-out regions later on
  columns_to_keep = bldgs_df.columns.to_list()
  additional = list(set([f'ADM{adm_no}_ES', 'ADM2_ES', 'ADM1_ES', 'ADM0_ES']))
  columns_to_keep = columns_to_keep + additional
  columns_to_keep.remove('location_geom')
  bldgs_df_2 = bldgs_df_2[columns_to_keep]

  return bounds_w_data, bldgs_df_2

mex_areas_2, mex_bldgs_2 = narrow_down(mex_bldgs, mex_areas, adm_no='2')
chl_areas_2, chl_bldgs_2 = narrow_down(chl_bldgs, chl_areas, adm_no='3')

In [ ]:
# Function to obtain Sentinel data for a specified region

# Data for 2020, because the infrastructure dataset (next cell) appears to be from 2021
#   - May be better to use an earlier year in the hope of excluding unregistered CAFOs,
#     which might be post-2021?
#   - May be better to restrict to a certain season
#   - May need to use better cloud masking:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

def get_sentinel(boundary, sentinel_bands):

  sentinel = (
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterDate('2020-01-01', '2020-12-31')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
      .select(sentinel_bands)
      .median() #crude cloud filter
      .clip(boundary)
  )

  return sentinel

In [ ]:
# Function to clip Sentinel data to an area around each farm and not-farm
# building

def extract_snippets(buildings_fc, sentinel_data, filename=None, folder=None):

  # For each polygon, define a square region around its centroid
  def buffer_and_bound(feature, buffer_radius=training_image_radius):
    return feature.centroid().buffer(buffer_radius, 2).bounds()

  areas = buildings_fc.map(buffer_and_bound)

  # Obtain Sentinel data for the polygon
  pix = sentinel_data.sampleRegions(
      collection=areas,
      scale=10,
      geometries=True)

  return areas, pix

In [ ]:
%%skip
# Get the Sentinel snippets for each location in each AMDx-level area, and write
# to file

def create_interim_files(bldgs, bounds, prefix, adm_no):

  for place in bounds[f'ADM{adm_no}_ES']:

    # Convert building info and boundary to ee featureCollections
    geometry = geemap.gdf_to_ee(bounds[bounds[f'ADM{adm_no}_ES']==place][['geometry']])
    bldgs_fc = geemap.gdf_to_ee(bldgs[bldgs[f'ADM{adm_no}_ES'] == place])

    # Get the Sentinel data for this area
    sentinel = get_sentinel(geometry, sentinel_bands)

    # Extract Sentinel data around each building location
    _, pix = extract_snippets(bldgs_fc, sentinel)

    # Save a file containing all the snippets for each region
    fname = unidecode(place).title().replace(" ", "").strip()
    try:
      #print(f"Saving {prefix}: {place} (-->{fname})")
      utils.write_to_file(pix, f"{prefix}_{fname}", 'interim_files')
    except:
      print(f" -- {place} failed")

create_interim_files(mex_bldgs_2, mex_areas_2, 'mex', adm_no='2')
create_interim_files(chl_bldgs_2, chl_areas_2, 'chl', adm_no='3')

In [ ]:
utils.ee_task_status(n_tasks=10)

Task 3HWMJAEAKIJWWXRTX3JTZ4H7 started at 2024-08-08 01:23:53.703000
Current status: COMPLETED
Task H36ME664KMRK76O4RV3DIA2P started at 2024-08-08 01:23:53.708000
Current status: COMPLETED
Task PT3OJIIL54DO2X2XBHGHRXZN started at 2024-08-08 01:23:45.911000
Current status: COMPLETED
Task Y3YMR72X6GMABQ5KIPCLDQIW started at 2024-08-08 01:23:39.304000
Current status: COMPLETED
Task UP2CBJDU3FBEVJK6BTIATA7P started at 2024-08-08 01:23:35.969000
Current status: COMPLETED
Task DEKV3LSJRR4V64HRHH7ZXVC7 started at 2024-08-08 01:23:35.968000
Current status: COMPLETED
Task W7ZGW4NIKYYEDWIXVRNTELGC started at 2024-08-08 01:23:32.874000
Current status: COMPLETED
Task HOUKQ2AFU3C3V4BJFNINFSFV started at 2024-08-08 01:23:26.321000
Current status: COMPLETED
Task QWINHWLIOWYO3PUKY57LB366 started at 2024-08-08 01:23:22.672000
Current status: COMPLETED
Task BFDMD4W7JZXH663BBFGXT2AO started at 2024-08-08 01:23:13.129000
Current status: COMPLETED


In [ ]:
# Convert pixel coords and values into a dictionary of 3D numpy array (height,
# width, channels). Band order is RGB, images are 64x64 pix, scaled to 0-255. (DTYPE??)

def create_images(gdf, index_start, verbose=False):

    # Remove suffixes from pixel ID numbers so we can group all pixels for a
    # given farm or not-farm
    # E.g, 1_1, 1_2, 1_3, 1_4 --> 1, 1, 1, 1

    gdf['id'] = gdf['id'].str.split('_').str[0]
    gdf['id'] = gdf['id'].astype(int)

    rejected = []
    arr_list = []

    # Here, a group will be an individual farm or not-farm
    groups = gdf.groupby(by='id')
    for n, group in groups:
      data = {'B4': [], 'B3': [], 'B2': []}
      # Identify image rows for each farm/not-farm
      _ = group.groupby(by=group.geometry.y)
      # Gather the group's pixels into a 3D array
      for coord, vals in _:
        for band in ['B4', 'B3', 'B2']:
          data[band].append([b for b in vals[band]])
      try:
        arr = np.stack([np.array(data['B4']), np.array(data['B3']), np.array(data['B2'])])
      except ValueError as e:
        # These are presumably groups that intersect with the boundary of the region
        # They cause problems because they aren't rectangular; might able to pad but
        # that seems like more trouble than it's worth
        rejected.append(group['id'].unique()[0])
        continue

      # Move the channels axis to the end
      arr = np.moveaxis(arr, [0], [2])

      # Resize the images to 64 x 64 pix
      arr = scipy.ndimage.zoom(
          arr,
          (64/arr.shape[0], 64/arr.shape[1], 1),
          mode='reflect'
          )

      # Rescale to 0-255 (using max over all bands)
      arr = (arr / np.max(arr)) * 255

      # The image is ready now, so add it to the dict
      arr_list.append(arr)

    # Create a version of the input array that has just one row per farm/not-farm
    # and doesn't include the now-redundant band info
    gdf = gdf[~gdf['id'].isin(rejected)]
    gdf = gdf.drop(columns=['B2', 'B3', 'B4']).drop_duplicates(subset=['id'])\
             .drop(columns=['id'])
    new_index = [i for i in range(index_start, index_start+len(gdf))]
    gdf.index = new_index

    # Make a dict of arrays in which keys are guaranteed to match gdf
    arr_dict = {key: value for key, value in zip(new_index, arr_list)}

    if verbose:
      print(f"Started with {len(groups)} farms/not-farms")
      print(f"Rejected {len(rejected)} images with irregular shapes")
      print(f'Retained {len(new_index)} images')

    return arr_dict, gdf

In [26]:
files = glob.glob(f'{interim_path}*.geojson')

gdf_list = []
index_start = 0
image_dict_list = []

for n, f in enumerate(files):
  gdf = gpd.read_file(f)
  image_dict, gdf = create_images(gdf, index_start)
  if len(gdf) == 0:
    # No images remain after rejecting irregular ones
    continue
  gdf_list.append(gdf)
  image_dict_list.append(image_dict)
  index_start += len(gdf)

main_gdf = pd.concat(gdf_list)
image_dict = image_dict_list[0]
for d in image_dict_list[1:]:
  image_dict.update(d)

 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Angamacutiro.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Cacahoatan.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_CerroDeSanPedro.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Chiautla.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Chimalhuacan.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Chucandiro.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Cuautitlan.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Cuzama.geojson
 -- No images remain for /content/drive/MyDrive/CAFO_data/forTraining/interim_files/mex_Kinchil.geojson
 -- No images remain for /content/dri

In [27]:
main_gdf.to_pickle(f"{out_path}metadata_gdf.pkl")
with open(f"{out_path}sentinel_images.pkl", 'wb') as f:
    pickle.dump(image_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
display(main_gdf.tail(2))

# Countries
for name in ['Mexico', 'Chile']:
  temp = main_gdf[(main_gdf['Dataset name'] == name)]
  print(f"\nThere are {len(temp)} entries from {name}")
  temp = main_gdf[(main_gdf['Dataset name'] == name) & (main_gdf['Farm type'].isin(["Broiler", "Layer", "Poultry", "Pig", "Dairy"]))]
  print(f" --{len(temp)} are farms")
  temp = main_gdf[(main_gdf['Dataset name'] == name) & (main_gdf['Farm type'] == 'Non-farm')]
  print(f" --{len(temp)} are not-farms \n")

# Animal types
for item in ["Broiler", "Layer", "Poultry"]:
  num = len(main_gdf[main_gdf['Farm type'] == item])
  print(f"There are {num} rows of type {item} in the dataset")
temp = main_gdf[main_gdf['Farm type'].isin(["Broiler", "Layer", "Poultry"])]
print(f" --> there are {len(temp)} poultry farms in total")
for item in ["Pig", "Non-farm"]:
  num = len(main_gdf[main_gdf['Farm type'] == item])
  print(f"There are {num} rows of type {item} in the dataset")


,ADM0_ES,ADM1_ES,ADM2_ES,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,ADM3_ES,Number of animals
5818,Chile,Región de Arica y Parinacota,Arica,6448.1678,2.218523,Chile,Poultry,166.970649,"POLYGON ((-70.275874 -18.382762, -70.275882 -1...",POINT (-70.27848 -18.38452),Arica,166456.0
5819,Chile,Región de Arica y Parinacota,Arica,4223.9741,1.516956,Chile,Non-farm,80.602010,None,POINT (-70.29950 -18.36107),Arica,NaN



There are 5004 entries from Mexico
 --3897 are farms
 --1037 are not-farms 


There are 816 entries from Chile
 --531 are farms
 --270 are not-farms 

There are 1380 rows of type Broiler in the dataset
There are 740 rows of type Layer in the dataset
There are 388 rows of type Poultry in the dataset
 --> there are 2508 poultry farms in total
There are 1920 rows of type Pig in the dataset
There are 1307 rows of type Non-farm in the dataset
